In [1]:
import numpy as np

from ase.io import read
from ase import units
from ase.calculators.aims import Aims
from ase.calculators.socketio import SocketIOCalculator

In [2]:
alias tail tail

In [3]:
atoms = read('si.in', format='aims')

In [4]:
command = 'orterun -n 4 /home/knoop/FHIaims/source/build_190115/aims.190101.mpi.x'
species_dir = '/home/knoop/FHIaims/aimsfiles/species_defaults/light'

aims_settings = {
    'command': command,
    'species_dir': species_dir,
    'output_level': 'MD_light',
    'relativistic': 'atomic_zora scalar',
    'xc': 'pw-lda',
    'k_grid': 3 * [2],
    'compute_heat_flux': True,
}

In [5]:
port = 12345
aims = Aims(**{'use_pimd_wrapper': f'localhost {port}', **aims_settings})

calc = SocketIOCalculator(aims, port = port)

In [6]:
atoms.calc = calc
atoms.calc.calculate(atoms)

In [7]:
tail aims.out

  lattice_vector         0.00000000        2.72000000        2.72000000
  lattice_vector         2.72000000        0.00000000        2.72000000
  lattice_vector         2.72000000        2.72000000        0.00000000

            atom         0.00000000        0.00000000        0.00000000  Si
            atom         1.36000000        1.36000000        1.36000000  Si
 ------------------------------------------------------------------------
  @ DRIVER MODE: Message from server: STATUS      
  @ DRIVER MODE: Message from server: GETFORCE    
  @ DRIVER MODE: Returning v,forces,stress 


In [8]:
def get_stresses(atoms):
    atoms.calc.server.protocol.sendmsg('GETSTRESSES')
    msg = atoms.calc.server.protocol.recvmsg()
    assert msg == 'STRESSREADY'
    natoms = atoms.calc.server.protocol.recv(1, np.int32)
    stresses = atoms.calc.server.protocol.recv((int(natoms), 3, 3), np.float64)
    stresses *= units.Ha
    return stresses

In [9]:
def socket_stress_off(atoms):
    calc.server.protocol.sendmsg('STRESSES_OFF')
    
def socket_stress_on(atoms):
    calc.server.protocol.sendmsg('STRESSES_ON')

In [10]:
get_stresses(atoms)

array([[[-8.18506461e-01,  8.11746092e-11, -3.83705232e-12],
        [ 8.11746092e-11, -8.18507265e-01,  3.98732356e-11],
        [-3.83705232e-12,  3.98732356e-11, -8.18507266e-01]],

       [[-8.18506461e-01, -4.18198127e-11, -5.45953548e-11],
        [-4.18198127e-11, -8.18507265e-01,  1.66355809e-11],
        [-5.45953548e-11,  1.66355809e-11, -8.18507266e-01]]])

In [11]:
tail aims.out

  lattice_vector         2.72000000        2.72000000        0.00000000

            atom         0.00000000        0.00000000        0.00000000  Si
            atom         1.36000000        1.36000000        1.36000000  Si
 ------------------------------------------------------------------------
  @ DRIVER MODE: Message from server: STATUS      
  @ DRIVER MODE: Message from server: GETFORCE    
  @ DRIVER MODE: Returning v,forces,stress 
  @ DRIVER MODE: Message from server: GETSTRESSES 
  @ DRIVER MODE: fkdev returning stress per atom 


In [12]:
socket_stress_off(atoms)

In [13]:
tail aims.out

            atom         0.00000000        0.00000000        0.00000000  Si
            atom         1.36000000        1.36000000        1.36000000  Si
 ------------------------------------------------------------------------
  @ DRIVER MODE: Message from server: STATUS      
  @ DRIVER MODE: Message from server: GETFORCE    
  @ DRIVER MODE: Returning v,forces,stress 
  @ DRIVER MODE: Message from server: GETSTRESSES 
  @ DRIVER MODE: fkdev returning stress per atom 
  @ DRIVER MODE: Message from server: STRESSES_OFF
  @ DRIVER MODE: fkdev switch stress and heat flux off 


In [14]:
atoms.calc.calculate(atoms, system_changes=['positions'])

In [15]:
tail aims.out

  lattice_vector         0.00000000        2.72000000        2.72000000
  lattice_vector         2.72000000        0.00000000        2.72000000
  lattice_vector         2.72000000        2.72000000        0.00000000

            atom         0.00000000        0.00000000        0.00000000  Si
            atom         1.36000000        1.36000000        1.36000000  Si
 ------------------------------------------------------------------------
  @ DRIVER MODE: Message from server: STATUS      
  @ DRIVER MODE: Message from server: GETFORCE    
  @ DRIVER MODE: Returning v,forces,stress 


In [16]:
get_stresses(atoms)

array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])

In [17]:
atoms.calc.close()

In [18]:
tail aims.out

          |   Average:        8.104 MB
          | Largest tracked array allocation (hamiltonian_shell):
          |   Minimum:        1.708 MB (on task 3)
          |   Maximum:        1.760 MB (on task 1)
          |   Average:        1.747 MB
          Note:  These values currently only include a subset of arrays which are explicitly tracked.
          The "true" memory usage will be greater.

          Have a nice day.
------------------------------------------------------------
